### Домашнее задание <a class="anchor" id="hw"></a><center>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

__Задача:__ подойдет ли репетитор для подготовки к экзамену по математике

In [2]:
X = np.array([ [   1,    1,  500,    1],
               [   1,    1,  700,    1],
               [   1,    2,  750,    2],
               [   1,    5,  600,    1],
               [   1,    3, 1450,    2],
               [   1,    0,  800,    1],
               [   1,    5, 1500,    3],
               [   1,   10, 2000,    3],
               [   1,    1,  450,    1],
               [   1,    2, 1000,    2]], dtype = np.float64)

y = np.array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1])

In [3]:
def standard_scale(X):
    mean = X.mean(axis=0)
    std = X.std(axis=0)
    return (X - mean) / std

In [5]:
X_st = X.copy().astype(np.float64)
X_st[:, 1:4] = standard_scale(X_st[:, 1:4])
X_st

array([[ 1.        , -0.70710678, -0.97958969, -0.89625816],
       [ 1.        , -0.70710678, -0.56713087, -0.89625816],
       [ 1.        , -0.35355339, -0.46401617,  0.38411064],
       [ 1.        ,  0.70710678, -0.77336028, -0.89625816],
       [ 1.        ,  0.        ,  0.97958969,  0.38411064],
       [ 1.        , -1.06066017, -0.36090146, -0.89625816],
       [ 1.        ,  0.70710678,  1.08270439,  1.66447944],
       [ 1.        ,  2.47487373,  2.11385144,  1.66447944],
       [ 1.        , -0.70710678, -1.08270439, -0.89625816],
       [ 1.        , -0.35355339,  0.05155735,  0.38411064]])

**1. *Измените функцию calc_logloss так, чтобы нули по возможности не попадали в np.log.**

1 метод

In [6]:
def calc_logloss(y, y_pred):
    y_pred = np.where(y_pred == 1, 1 - 1e-8, y_pred)
    y_pred = np.where(y_pred == 0, 0 + 1e-8, y_pred)
    err = - np.mean(y * np.log(y_pred) + (1.0 - y) * np.log(1.0 - y_pred))
    return err

In [7]:
eps = 1e-8

np.clip([1, 0, 0], eps, 1 - eps)

array([9.9999999e-01, 1.0000000e-08, 1.0000000e-08])

In [8]:
def calc_logloss(y, y_pred):
    eps = 1e-8
    y_pred = np.clip(y_pred, eps, 1 - eps)
    err = - np.mean(y * np.log(y_pred) + (1.0 - y) * np.log(1.0 - y_pred))
    return err

2 метод

In [9]:
def calc_logloss(y, y_pred):
    err = 0
    for i in range(len(y)):
        if y_pred[i] == 0:
            err += (1.0 - y[i]) * np.log(1.0 - y_pred[i])
        elif y_pred[i] == 1:
            err += y[i] * np.log(y_pred[i])
        else:
            err += y[i] * np.log(y_pred[i]) + (1.0 - y[i]) * np.log(1.0 - y_pred[i])
    return - err / len(y)

In [10]:
y1 = np.array([1, 0])
y_pred1 = np.array([0.9, 0])
calc_logloss(y1, y_pred1)

0.05268025782891314

In [11]:
y1 = np.array([1, 0])
y_pred1 = np.array([1, 0.1])
calc_logloss(y1, y_pred1)

0.05268025782891314

In [12]:
y1 = np.array([1, 0])
y_pred1 = np.array([0.9, 0.1])
calc_logloss(y1, y_pred1)

0.10536051565782628

**2. Подберите аргументы функции eval_model для логистической регрессии таким образом, чтобы log loss был минимальным.**

In [13]:
def sigmoid(z):
    res = 1 / (1 + np.exp(-z))
    return res

In [14]:
def eval_model(X, y, iterations, eta=1e-4):
    np.random.seed(42)
    W = np.random.randn(X.shape[1])
    n = X.shape[0]
    
    for i in range(iterations):
        z = np.dot(X, W)
        y_pred = sigmoid(z)
        err = calc_logloss(y, y_pred)
        
        dQ = 1/n * X.T @ (y_pred - y)
        W -= eta * dQ
        if i % (iterations / 10) == 0:
            print(i, W, err)
            
    final_error = calc_logloss(y, y_pred)
    return W, final_error

1 метод

аналог GridSearch

In [15]:
from tqdm import tqdm

In [17]:
iterations = np.logspace(2, 5, 4, dtype=np.int)
etas = np.linspace(1e-2, 5, 10)

best_error = np.inf
best_params = {}

for iteration in tqdm(iterations):
    for eta in etas:
        W, error = eval_model(X_st, y, iterations=iteration, eta=eta)
        if error < best_error:
            best_error = error
            best_params = {
                'iteration': iteration,
                'eta': eta
            }

print(f'Ошибка {best_error} при параметрах {best_params}')

  0%|                                                                                            | 0/4 [00:00<?, ?it/s]

0 [ 0.49635916 -0.1409725   0.64535427  1.52178506] 0.760958797591889
10 [ 0.49265881 -0.16787071  0.62221328  1.50954192] 0.7467006818024328
20 [ 0.48868992 -0.19442917  0.59944411  1.49767566] 0.7328651153682048
30 [ 0.48446029 -0.22063955  0.57705409  1.48619252] 0.7194524935754576
40 [ 0.47997869 -0.24649338  0.55505065  1.4750986 ] 0.7064630339679188
50 [ 0.47525478 -0.27198192  0.53344129  1.46439984] 0.6938967869573868
60 [ 0.47029912 -0.29709626  0.51223358  1.45410204] 0.6817536325961876
70 [ 0.46512312 -0.32182725  0.49143512  1.44421083] 0.6700332616838689
80 [ 0.45973898 -0.34616556  0.47105359  1.4347317 ] 0.658735139989009
90 [ 0.45415964 -0.37010173  0.4510966   1.42566993] 0.6478584551577349
0 [ 0.47667685 -0.29112687  0.51593213  1.45276794] 0.760958797591889
10 [ 0.15302827 -1.07349673 -0.05330995  1.447242  ] 0.4588786685427442
20 [ 0.0338664  -1.33389885 -0.1641304   1.79467485] 0.41846490870843833
30 [-2.07371606e-03 -1.53135455e+00 -2.56777284e-01  2.10047940e+00]

 25%|█████████████████████                                                               | 1/4 [00:00<00:01,  2.79it/s]

 [ 0.17936395 -2.82458553 -1.36927006  4.74462751] 0.27825805572871576
60 [ 0.23875942 -2.96699583 -1.55395937  5.11620862] 0.26931659090725035
70 [ 0.29672835 -3.09417692 -1.72366704  5.45548151] 0.26188447188325964
80 [ 0.35290478 -3.21095954 -1.88128838  5.77047161] 0.255494053822906
90 [ 0.40719937 -3.32027404 -2.02897313  6.06648602] 0.24986274233944156
0 [ 0.39794758 -0.89174437 -0.00175644  1.17669946] 0.760958797591889
10 [-0.01353507 -1.89519528 -0.45686524  2.70566377] 0.3577725442965939
20 [ 0.03621765 -2.3383901  -0.81849901  3.58537057] 0.314934706141088
30 [ 0.10660652 -2.62004956 -1.11813471  4.23146058] 0.2927433926819891
40 [ 0.18143823 -2.83111703 -1.37586041  4.75941413] 0.27813263679926564
50 [ 0.25550972 -3.00545132 -1.60379817  5.21687129] 0.2672390433364995
60 [ 0.32711449 -3.15841753 -1.80956     5.6276333 ] 0.25849632049319815
70 [ 0.39579449 -3.29781006 -1.99822733  6.00510388] 0.2511399533730289
80 [ 0.46155658 -3.42789533 -2.17335216  6.35748839] 0.244748004

700 [ 2.48736604 -7.7868268  -7.3091592  17.45406699] 0.1341292088914971
800 [ 2.66815675 -8.21596505 -7.7911034  18.52503036] 0.12838041973375916
900 [ 2.83202648 -8.6086002  -8.23284395 19.50669159] 0.12355607983650319
0 [ 0.37826527 -1.04189875 -0.13117859  1.10768234] 0.760958797591889
100 [ 0.70140754 -3.8967646  -2.78610189  7.61213821] 0.2248184328439317
200 [ 1.25588979 -5.02435647 -4.15919242 10.52750401] 0.18851113578673367
300 [ 1.67265871 -5.92285194 -5.20190754 12.79714775] 0.16672163177075466
400 [ 2.00685651 -6.67187117 -6.05418953 14.67275429] 0.15191012562883693
500 [ 2.28573584 -7.31405177 -6.77808771 16.27517862] 0.14113268685296632
600 [ 2.52528107 -7.87643933 -7.40977167 17.67759335] 0.13289671205266354
700 [ 2.73562922 -8.3772419  -7.97241164 18.92798778] 0.12636114254086359
800 [ 2.92356247 -8.82922424 -8.48169578 20.05941348] 0.12101769706627168
900 [ 3.09381296 -9.24163289 -8.94869163 21.09549951] 0.11654212304850761
0 [ 0.35858295 -1.19205313 -0.26060073  1.03

...

0 [ 0.49635916 -0.1409725   0.64535427  1.52178506] 0.760958797591889
1000 [ 0.06528341 -1.25315687 -0.13031431  1.68209805] 0.4264242191110676
2000 [-0.00591593 -1.60120112 -0.2960273   2.21707777] 0.3819609701086998
3000 [-0.01097467 -1.85119187 -0.44498757  2.64578105] 0.35499278688362434
4000 [-2.64430856e-04 -2.04430387e+00 -5.82222091e-01  3.00431189e+00] 0.33646376256331917
5000 [ 0.01719932 -2.20052369 -0.70941905  3.31414696] 0.3227497767438226
6000 [ 0.03850056 -2.33129977 -0.82799326  3.58846186] 0.31204928819257577
7000 [ 0.06216276 -2.44374222 -0.93914508  3.83587188] 0.30336396882449485
8000 [ 0.08727955 -2.54253835 -1.04387339  4.06228822] 0.2960963045725309
9000 [ 0.11325547 -2.63091817 -1.14300269  4.27192266] 0.28986691616198357
0 [ 0.47667685 -0.29112687  0.51593213  1.45276794] 0.760958797591889
1000 [ 1.09439377 -4.68780498 -3.75952282  9.668002  ] 0.19782429297287785
2000 [ 1.8020679  -6.21006696 -5.53005669 13.51756994] 0.16059718650418509
3000 [ 2.29969558 -7.34

 75%|███████████████████████████████████████████████████████████████                     | 3/4 [00:26<00:07,  7.75s/it]

0 [ 0.49635916 -0.1409725   0.64535427  1.52178506] 0.760958797591889
10000 [ 0.13968874 -2.71118375 -1.23721448  4.46786924] 0.28442328798629035
20000 [ 0.39628294 -3.29726272 -1.99976528  6.00637336] 0.2504398670404289
30000 [ 0.61960886 -3.7363256  -2.58029515  7.18657544] 0.23065655399133544
40000 [ 0.81489617 -4.12126594 -3.06873794  8.20226418] 0.21607222957970404
50000 [ 0.98919528 -4.47219897 -3.49975648  9.11347336] 0.20436661619311464
60000 [ 1.14704184 -4.79680837 -3.88964341  9.94707834] 0.19458887365606037
70000 [ 1.29144663 -5.0994097  -4.24742866 10.71823126] 0.18623385134315354
80000 [ 1.42456227 -5.38299036 -4.57885232 11.43691088] 0.17898618252823809
90000 [ 1.54802115 -5.64984567 -4.88795526 12.11039037] 0.17262828235969302
0 [ 0.47667685 -0.29112687  0.51593213  1.45276794] 0.760958797591889
10000 [  4.08642504 -11.6647506  -11.79181561  27.32393823] 0.09579059139566205
20000 [  5.43283404 -14.80987058 -16.03275496  36.15953504] 0.07645710569198655
30000 [  6.445550

100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [03:46<00:00, 56.54s/it]

Ошибка 0.003985684510654154 при параметрах {'iteration': 100000, 'eta': 5.0}


2 метод

Вручную

In [18]:
W, error = eval_model(X_st, y, iterations=100, eta=4)

0 [ 0.35471749 -1.22154236 -0.28601831  1.02511075] 0.760958797591889
10 [ 1.74524510e-03 -2.28338321e+00 -6.79637042e-01  3.22729351e+00] 0.33223690107609094
20 [ 0.10463396 -2.65190818 -1.1068567   4.25259661] 0.29290854740108785
30 [ 0.21466455 -2.92848549 -1.47776456  4.9846682 ] 0.2731021569812502
40 [ 0.32011366 -3.15150546 -1.78839465  5.59488481] 0.2595299652045495
50 [ 0.41898795 -3.34761692 -2.05973011  6.13321495] 0.2490584530186956
60 [ 0.51162553 -3.5276709  -2.30361599  6.62353031] 0.24042303880559396
70 [ 0.59877762 -3.69672027 -2.52725542  7.07891825] 0.23300566324010216
80 [ 0.68116596 -3.85740865 -2.7352801   7.50725774] 0.22646377060006245
90 [ 0.7593943  -4.01127147 -2.9307987   7.91363943] 0.22058939385110884


**3. Создайте функцию calc_pred_proba, возвращающую предсказанную вероятность класса 1 (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred_proba).**

In [19]:
def calc_pred_proba(W, X):
    y_pred_proba = sigmoid(np.dot(X, W))
    return y_pred_proba

In [ ]:
calc_pred_proba(W, X_st)

**4. Создайте функцию calc_pred, возвращающую предсказанный класс (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred).**

In [ ]:
def calc_pred(W, X):
    y_pred_proba = calc_pred_proba(W, X)
    y_pred = np.where(y_pred_proba > 0.5, 1, 0)
    return y_pred

In [ ]:
pred = calc_pred(W, X_st)
pred

**5. *Реализуйте функции для подсчета Accuracy, матрицы ошибок, точности и полноты, а также F1 score.**

*Accuracy*

In [ ]:
def accuracy(y, y_pred):
    accuracy = np.mean(y == y_pred)
    return accuracy

In [ ]:
print(f'True \t  {y}')
print(f'Predicted {pred}')

In [ ]:
accuracy(y, pred)

*Confusion Matrix*

In [ ]:
# TP  FP
# FN  TN

In [ ]:
def confusion_matrix(y, y_pred):
    cm = np.zeros((2, 2))
    for i in range(len(y)): 
        if y[i] == y_pred[i] == 1: # TP
            cm[0][0] += 1
        elif y[i] == y_pred[i] == 0: # TN
            cm[1][1] += 1
        elif y[i] != y_pred[i] and y[i] == 1: # FN
            cm[1][0] += 1
        elif y[i] != y_pred[i] and y[i] == 0: # FP
            cm[0][1] += 1
    return cm

In [ ]:
cm = confusion_matrix(y, pred)
cm

*Precision*

In [ ]:
def precision(y, y_pred):
    cm = confusion_matrix(y, y_pred)
    TP = cm[0][0]
    TN = cm[1][1]
    FP = cm[0][1]
    FN = cm[1][0]
    precision = TP / (TP + FP)
    return precision

In [ ]:
precision(y, pred)

*Recall*

In [ ]:
def recall(y, y_pred):
    cm = confusion_matrix(y, y_pred)
    TP = cm[0][0]
    TN = cm[1][1]
    FP = cm[0][1]
    FN = cm[1][0]
    recall = TP / (TP + FN)
    return recall

In [ ]:
recall(y, pred)

*F-score*

In [ ]:
def f_score(y, y_pred):
    pr = precision(y, y_pred)
    rec = recall(y, y_pred)
    f_score = 2 * pr * rec / (pr + rec)
    return f_score

In [ ]:
f_score(y, pred)

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix


print(f'accuracy {accuracy_score(y, pred)}')
print(f'precision {precision_score(y, pred)}')
print(f'recall {recall_score(y, pred)}')
print(f'f-score {f1_score(y, pred)}')
print(f'\n {confusion_matrix(y, pred)}')

**6. Могла ли модель переобучиться? Почему?**

Могла, если взять большое количество итераций, когда ошибка будет стремиться к нулю.